# Movielens 영화 추천 실습

## (1) 데이터 준비, 전처리

In [1]:
# 1) wget으로 데이터 다운로드
# $ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

# 2) 다운받은 데이터를 작업디렉토리로 옮김
# $ mv ml-1m.zip ~/aiffel/recommendata_iu/data

# 3) 작업디렉토리로 이동
# $ cd ~/aiffel/recommendata_iu/data

# 4) 압축 해제
# $ unzip ml-1m.zip

In [2]:
import os
import numpy as np
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


* 3점 이상만 남깁니다.

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## (2) MF모델구성하여 분석해보기

* ratings에 있는 유니크한 영화 개수
* rating에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [8]:
# 검색편하게 소문자로
movies['title'] = movies.title.str.lower()

* ratings에 있는 유니크한 영화 개수

In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    836478 non-null  int64
 1   movie_id   836478 non-null  int64
 2   count      836478 non-null  int64
 3   timestamp  836478 non-null  int64
dtypes: int64(4)
memory usage: 31.9 MB


In [10]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

In [14]:
# info의 갯수와 같으므로 다 고유한 영화이
print(ratings.movie_id.unique())
print(ratings.movie_id.nunique())
print(ratings.user_id.nunique())

# 3628개로 movies의 영화갯수 3883와 좀 다르다.

[1193  661  914 ...  690 2909 1360]
3628
6039


* rating에 있는 유니크한 사용자 수

In [15]:
ratings.keys()

Index(['user_id', 'movie_id', 'count', 'timestamp'], dtype='object')

* 가장 인기 있는 영화 30개(인기순)

In [16]:
movie_count = ratings.groupby('movie_id')['count'].count()
m_count = movie_count.sort_values(ascending=False).head(30)
m_count

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count, dtype: int64

# (3) 내가 좋아하는 영화 5개 추가하기

In [17]:
my_movie = ['gladiator','die hard','mask','terminator 2', 'home alone']

In [18]:
sorted(ratings.movie_id.unique())[-1]

3952

In [19]:
ratings.movie_id.count()

836478

In [20]:
movies.keys()

Index(['movie_id', 'title', 'genre'], dtype='object')

In [21]:
for i,movie in movies.title.items():
    for j in my_movie:
        if j in movie:
            print(i)
            print(movie)
            print(movies.movie_id.loc[i])
            print('---')
        

163
die hard: with a vengeance (1995)
165
---
363
mask, the (1994)
367
---
582
home alone (1990)
586
---
585
terminator 2: judgment day (1991)
589
---
1023
die hard (1988)
1036
---
1349
die hard 2 (1990)
1370
---
1659
home alone 3 (1997)
1707
---
1740
man in the iron mask, the (1998)
1801
---
1937
mask of zorro, the (1998)
2006
---
2540
king of masks, the (bian lian) (1996)
2609
---
2556
black mask (hak hap) (1996)
2625
---
2884
home alone 2: lost in new york (1992)
2953
---
3144
batman: mask of the phantasm (1993)
3213
---
3509
gladiator (2000)
3578
---


* 다이하드  
인덱스 163
무비id 165

In [22]:
movies['title'].loc[163]

'die hard: with a vengeance (1995)'

In [23]:
movies['movie_id'].loc[163]

165

In [24]:
### id순으로 정렬했을 때 마지막 값은 6040
### 고유값의 갯수는 6039
### 1개는 어디?

In [25]:
ratings['user_id'].sort_values()

0             1
29            1
30            1
31            1
32            1
           ... 
999984     6040
999982     6040
999981     6040
999989     6040
1000208    6040
Name: user_id, Length: 836478, dtype: int64

In [26]:
ratings.user_id.nunique()

6039

In [27]:
# my_movie = ['gladiator','die hard','mask','terminator 2', 'home alone']

my_list = pd.DataFrame({'user_id':[6041]*5, 'movie_id':[3578, 165, 367, 589, 586], 'count':[5,4,5,5,4]})
my_list

,user_id,movie_id,count
0,6041,3578,5
1,6041,165,4
2,6041,367,5
3,6041,589,5
4,6041,586,4


In [28]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [29]:
ratings1 = ratings.drop('timestamp', axis=1)
ratings1.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [30]:
type(ratings1)
print(ratings1.shape)

(836478, 3)


In [31]:
type(my_list)
print(my_list.shape)

(5, 3)


In [32]:
datas = pd.concat([ratings1, my_list])
datas

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,3578,5
1,6041,165,4
2,6041,367,5
3,6041,589,5


# (4) CSR matrix 만들기

In [33]:
from scipy.sparse import csr_matrix
import numpy as np
from implicit.als import AlternatingLeastSquares

In [34]:
ratings.user_id.nunique()

6039

In [35]:
m_user = max(datas.user_id)
m_movie =max(datas.movie_id)
m_user, m_movie

(6041, 3952)

In [36]:
csr_data = csr_matrix((datas['count'],(datas.user_id, datas.movie_id)),shape=(6042,3953))

## (5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [37]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [38]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [39]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [40]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## (6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [41]:
# 작업방식이 선호도 유사영화

In [45]:
datas

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,3578,5
1,6041,165,4
2,6041,367,5
3,6041,589,5


* 내가 선호하는 5가지 영화중 1개에 대한 모델의 예측 선호도

In [47]:
jin_vec, gladi = als_model.user_factors[6041], als_model.item_factors[3578]

In [48]:
np.dot(jin_vec, gladi)

0.5596368

* 특정 영화1에 대한 선호도

In [52]:
my_movie1 = ['amageddon','speed']

In [53]:
for i,movie in movies.title.items():
    for j in my_movie1:
        if j in movie:
            print(i)
            print(movie)
            print(movies.movie_id.loc[i])
            print('---')
        

373
speed (1994)
377
---
1517
speed 2: cruise control (1997)
1556
---
1675
full speed (1996)
1724
---


In [54]:
jin_vec1, speed = als_model.user_factors[6041], als_model.item_factors[373]

In [55]:
np.dot(jin_vec1, speed)

0.011715835

In [ ]:
# speed 재밌게 봤는데 추천도가 매우 낮게 나왔다. 다이하드도 이런부류의 영화인데 학습이 잘못된 것일까.

## (7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [56]:
# filter_already_liked_items : 이미 평가한 아이템은 제외
movie_recommend = als_model.recommend(6041, csr_data, N=20, filter_already_liked_items=True)

In [57]:
movie_recommend

[(2571, 0.334351),
 (3753, 0.2769268),
 (3623, 0.2607184),
 (1240, 0.26057643),
 (480, 0.25172198),
 (110, 0.23914272),
 (3793, 0.23161224),
 (1580, 0.22909456),
 (457, 0.22329825),
 (2628, 0.21963781),
 (1610, 0.21247724),
 (317, 0.2067323),
 (648, 0.20525569),
 (2916, 0.20090187),
 (2762, 0.20035647),
 (1370, 0.19784582),
 (3489, 0.18994123),
 (3751, 0.18870841),
 (500, 0.18385166),
 (1200, 0.17789751)]

In [62]:
for i in movie_recommend:
    
    vv = movies[movies['movie_id']==i[0]]
    print(vv['title'].values)

['matrix, the (1999)']
['patriot, the (2000)']
['mission: impossible 2 (2000)']
['terminator, the (1984)']
['jurassic park (1993)']
['braveheart (1995)']
['x-men (2000)']
['men in black (1997)']
['fugitive, the (1993)']
['star wars: episode i - the phantom menace (1999)']
['hunt for red october, the (1990)']
['santa clause, the (1994)']
['mission: impossible (1996)']
['total recall (1990)']
['sixth sense, the (1999)']
['die hard 2 (1990)']
['hook (1991)']
['chicken run (2000)']
['mrs. doubtfire (1993)']
['aliens (1986)']


In [ ]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
# idx_to_artist = {v:k for k,v in artist_to_idx.items()}
# [idx_to_artist[i[0]] for i in similar_artist]

# idx_to_movie = {v:k for k,v in movie['title'].items()}
# [idx_to_movie[i[0]] for i in movies] 식으로 변형해봤으나 에러 발생해서 다른 방법진행

## (8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [64]:
datas

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,3578,5
1,6041,165,4
2,6041,367,5
3,6041,589,5


In [66]:
user = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
mo_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
mo_recommended

[(2571, 0.334351),
 (3753, 0.2769268),
 (3623, 0.2607184),
 (1240, 0.26057643),
 (480, 0.25172198),
 (110, 0.23914272),
 (3793, 0.23161224),
 (1580, 0.22909456),
 (457, 0.22329825),
 (2628, 0.21963781),
 (1610, 0.21247724),
 (317, 0.2067323),
 (648, 0.20525569),
 (2916, 0.20090187),
 (2762, 0.20035647),
 (1370, 0.19784582),
 (3489, 0.18994123),
 (3751, 0.18870841),
 (500, 0.18385166),
 (1200, 0.17789751)]

In [67]:
for i in movie_recommend:
    
    vv = movies[movies['movie_id']==i[0]]
    print(vv['title'].values)

['matrix, the (1999)']
['patriot, the (2000)']
['mission: impossible 2 (2000)']
['terminator, the (1984)']
['jurassic park (1993)']
['braveheart (1995)']
['x-men (2000)']
['men in black (1997)']
['fugitive, the (1993)']
['star wars: episode i - the phantom menace (1999)']
['hunt for red october, the (1990)']
['santa clause, the (1994)']
['mission: impossible (1996)']
['total recall (1990)']
['sixth sense, the (1999)']
['die hard 2 (1990)']
['hook (1991)']
['chicken run (2000)']
['mrs. doubtfire (1993)']
['aliens (1986)']


In [ ]:
# 12개의 영화가 추천창에 떴다면 생각해볼만한 영화이고, 7개의 영화가 재미있게 봤던 영화들이다, 추천성능이 괜찮은것 같다.

In [ ]:
### 잘 이해 안되는 부분들####

In [ ]:
# # 고유한 유저, 아티스트를 찾아내는 코드
# user_unique = data['user_id'].unique()
# artist_unique = data['artist'].unique()

# # 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
# user_to_idx = {v:k for k,v in enumerate(user_unique)}
# artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [ ]:
# # 인덱싱이 잘 되었는지 확인해 봅니다. 
# print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
# print(artist_to_idx['black eyed peas'])

In [ ]:
# # indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# # dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# # user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# # 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
# temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
# if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
#     print('user_id column indexing OK!!')
#     data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
# else:
#     print('user_id column indexing Fail!!')

# # artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
# temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
# if len(temp_artist_data) == len(data):
#     print('artist column indexing OK!!')
#     data['artist'] = temp_artist_data
# else:
#     print('artist column indexing Fail!!')

# data

# # data의 user_id와 artist 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었나요? 이것으로 훈련을 위한 전처리가 완료되었습니다!!

* 이부분의 필요성

In [ ]:
# #아티스트의 id, 유사도) Tuple 로 반환하고 있습니다. 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜 주겠습니다.
# #artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
# idx_to_artist = {v:k for k,v in artist_to_idx.items()}
# [idx_to_artist[i[0]] for i in similar_artist]

In [ ]:
# 이 과정들을 영화추천에 적용하려하니 코드가 꼬여서, 없이 진행
# index:user_id >> user_id:index 변환  모델작업후 user_id:index>>index:user_id
# index:title >> title:index 변환  모델작업후 title:index>>index:title
# 모델작업후 추천적용시킬때 id로 나온것을 title로 전환?